In [1]:
#Imports
import json
import numpy as np
import pandas as pd
import csv
import os
from math import log
from pylab import lstsq
from pylab import matrix
from pylab import exp

In [2]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results,photo_name):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result_' + photo_name + '.json',"w") as miarch:
        miarch.write(json.dumps(results, cls=NumpyEncoder))

In [3]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open(path,'r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [4]:
def slicesParser_max_peaksI(dictres):
#extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x) for x in range(0,len(allSlices[0]['max_peaks_intensities']))]    
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    for i in range(0,len(allSlices)):
        df_sum.loc[i] = allSlices[i]['max_peaks_intensities'][0:] 
    return df_sum

In [5]:
def slicesParser_halfpeakstime(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_half_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_half_peaks']
    return df_sum

In [6]:
def slicesParser_amplitudes(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['amplitudes']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['amplitudes']

    return df_sum

In [7]:
def slicesParser_min_peaksI(dictres):
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['min_peaks_intensities']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['min_peaks_intensities']

    return df_sum

In [8]:
def slicesParser_peaktime(dictres):
    #extract all the slices data
    allSlices = dictres['slices']

    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_peaks']

    return df_sum

# Calcula el alternance ratio

AR_list = []
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/jsons/'
jsons_list = os.listdir(path)
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    slices = dic['slices']
    i = 0
    df = pd.DataFrame(columns=['transient1','transient2','transient_max'])
    print (file)
    for i in range(0,33):
        try:
            amps = slices[i]['amplitudes'][0:2]
            amps.append(max(amps))
            i += 1
            df.loc[len(df)] = amps
        except IndexError:
            pass
    df.to_csv(path + file + 'max_amp.csv', sep='\t')
print('Done')

In [12]:
# Guarda en tablas separadas los datos de cada json por parámetro

param_list = ['max_peaks_positions','min_peaks_positions','max_peaks_intensities','min_peaks_intensities','amplitudes','times_to_peaks','times_to_half_peaks']
path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/jsons/'
path2 = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/'
for param in param_list:
    os.makedirs(path2 + param + '/')
    jsons_list = os.listdir(path)
    for file in jsons_list:
        path_file = path + file
        dic = jasonReader(path_file)
        slices = dic['slices']
        i = 0
        df = pd.DataFrame(columns=['transient1','transient2'])
        print (file)
        for i in range(0,33):
            try:
                amps = slices[i][param][0:2]
                i += 1
                df.loc[len(df)] = amps
            except IndexError:
                pass
        df.to_csv(path2 + param + '/' + file + param + '.csv', sep='\t')
    print('Done')

010719ctrol1_analysis_result_ctrol1a0.json
010719ctrol1_analysis_result_ctrol1b0.json
010719ctrol8_analysis_result_ctrol8a0.json
010719ctrol8_analysis_result_ctrol8b0.json
010719Dan2_analysis_result_dan2a000.json
010719Dan2_analysis_result_dan2b000.json
010719Dan6_analysis_result_dan6a000.json
010719Dan6_analysis_result_dan6b000.json
010719VK3_analysis_result_vk3a000.json
010719VK3_analysis_result_vk3b002.json
010719VK7_analysis_result_vk7a000.json
010719VK7_analysis_result_vk7b000.json
010819c1_analysis_result_c1a000.json
010819c1_analysis_result_c1b000.json
010819c2_analysis_result_c2a000.json
010819c2_analysis_result_c2b000.json
010819c3_analysis_result_c3a000.json
010819c3_analysis_result_c3b001.json
010819c3_analysis_result_c3c000.json
031019c1_analysis_result_c1b000.json
031019c1_analysis_result_c1e000.json
031019c2_analysis_result_c2b000.json
031019c2_analysis_result_c2e000.json
031019c2_analysis_result_c2f000.json
031019c3_analysis_result_c3b000.json
031019c3_analysis_result_c3

In [13]:
# Guarda en una tabla los datos de la célula entera

path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/jsons/'
path2 = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/'
jsons_list = os.listdir(path)
os.makedirs(path2 + 'whole_cell/')
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['image']
    pairs = {k: dic[k] for k in list(dic)[1:]}
    keys = list(pairs.keys())
    df = pd.DataFrame()
    for label in range(0,len(pairs)):
        df[keys[label]] = pairs[keys[label]][0:2]
    df = df.T
    df.columns = ['Transient1', 'Transient2']
    df.to_csv(path2 + 'whole_cell/' + file + '.csv', sep='\t')
print('Done')

Done


In [79]:
# Cálculo del TAU67%

#     '''This function finds a tList in sec yList - measurements ySS - the steady state value of y returns amplitude of exponent tau - the time constant'''

def fitExponent(tList,yList,ySS=0):
    bList = [log(max(y-ySS,1e-6)) for y in yList]
    (w,residuals,rank,sing_vals) = lstsq(matrix([[1,t] for t in tList]),matrix(bList).T, rcond=None)
    tau = -1.0/w[1,0]
    amplitude = exp(w[0,0])
    return (amplitude,tau)

path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/jsons/'
path2 = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/'
jsons_list = os.listdir(path)
os.makedirs(path2 + 'taus_preliminar/')
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['slices']
    TAU = pd.DataFrame()
    for peak in range  (0, len(dic)):
        amp = ((np.array(dic[peak]['max_peaks_intensities'][0:2]) - np.array(dic[peak] ['min_peaks_intensities'][1:3]))/np.exp(1))+np.array(dic[peak] ['min_peaks_intensities'][1:3])
        tau_peaks = []
        for i in range(0,len(amp)):
            array = dic[peak]['intensities']
            value = amp[i]
            trans_list = dic[peak]['intensities'][dic[peak]['max_peaks_positions'][i]:dic[peak]['min_peaks_positions'][i+1]]
            x = np.array(range(0,len(trans_list)))
            try:
                (amplitudeEst,tauEst) = fitExponent(x,trans_list)
                yEst = amplitudeEst*(exp(-x/tauEst))
                tau_peaks.append(-tauEst*log(value/amplitudeEst))
            except:
                tau_peaks.append('nan')
        TAU[peak] = tau_peaks
    taus = pd.DataFrame(TAU.T.values, columns = ['transient1','transient2'])
    taus.to_excel(path2 + 'taus_preliminar/' + file + '.xlsx', engine='xlsxwriter')  

C:\Users\Leand\PyMOL\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  


In [88]:
# Cálculo del TAU67% whole cell

#     '''This function finds a tList in sec yList - measurements ySS - the steady state value of y returns amplitude of exponent tau - the time constant'''

def fitExponent(tList,yList,ySS=0):
    bList = [log(max(y-ySS,1e-6)) for y in yList]
    (w,residuals,rank,sing_vals) = lstsq(matrix([[1,t] for t in tList]),matrix(bList).T, rcond=None)
    tau = -1.0/w[1,0]
    amplitude = exp(w[0,0])
    return (amplitude,tau)

path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/jsons/'
path2 = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Luis/analisis/'
jsons_list = os.listdir(path)
# os.makedirs(path2 + 'taus_preliminar/')
TAU = pd.DataFrame()
for file in jsons_list:
    path_file = path + file
    dic = jasonReader(path_file)
    dic = dic['image']
    amp = ((np.array(dic['max_peaks_intensities'][0:2]) - np.array(dic['min_peaks_intensities'][1:3]))/np.exp(1))+np.array(dic['min_peaks_intensities'][1:3])
    tau_peaks = []
    for i in range(0,len(amp)):
        array = dic['intensities']
        value = amp[i]
        trans_list = dic['intensities'][dic['max_peaks_positions'][i]:dic['min_peaks_positions'][i+1]]
        x = np.array(range(0,len(trans_list)))
        try:
            (amplitudeEst,tauEst) = fitExponent(x,trans_list)
            yEst = amplitudeEst*(exp(-x/tauEst))
            tau_peaks.append(-tauEst*log(value/amplitudeEst))
        except:
            tau_peaks.append('nan')
    TAU[file] = tau_peaks
taus = pd.DataFrame(TAU.T.values, columns = ['transient1','transient2'])
taus.to_excel(path2 + 'wc_tau_preliminar' + '.xlsx', engine='xlsxwriter')  

In [12]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/taus_preliminar/'
experiments = os.listdir(path)
taus_exp = pd.DataFrame()
for i in range(0,len (experiments)):
    experiment = experiments[i]
    tau_file = pd.read_excel(path + experiment)
    del tau_file['Unnamed: 0']
    taus_exp[experiment] = list(tau_file.mean())
local_taus_mean = taus_exp.T
local_taus_mean.columns = ['transient1','transient2']

In [94]:
tau_file = pd.read_excel('/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/wc_tau_preliminar.xlsx')

In [27]:
tabulado = pd.read_csv('/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/tabulado')

In [23]:
local_taus_mean.head()

,transient1,transient2
311019vk2_analysis_result_vk2a000.json.xlsx,35.572060,32.934367
010819c1_analysis_result_c1b000.json.xlsx,18.855498,18.190851
311019vk2_analysis_result_vk2d000.json.xlsx,15.500993,15.722178
010719Dan6_analysis_result_dan6a000.json.xlsx,20.444386,-163.798429
060320dan6_analysis_result_d6d001.json.xlsx,16.850763,15.925080


In [96]:
tau_file ['index'] = local_taus_mean.index.tolist()
# tau_file.set_index('index', inplace=True)
del tau_file['Unnamed: 0']

In [97]:
tau_file.head()

,transient1,transient2,index
0,56.642803,50.441796,311019vk2_analysis_result_vk2a000.json.xlsx
1,21.848068,22.281768,010819c1_analysis_result_c1b000.json.xlsx
2,17.606736,17.085831,311019vk2_analysis_result_vk2d000.json.xlsx
3,108.615258,95.514131,010719Dan6_analysis_result_dan6a000.json.xlsx
4,65.503319,55.030121,060320dan6_analysis_result_d6d001.json.xlsx


In [29]:
tabulado.head()

,Unnamed: 0,experimento,célula,foto,tratamiento,frecuencia
0,0,010719SEL,010719ctrol1,ctrol1b0.tif,C,1
1,1,010719SEL,010719ctrol1,ctrol1a0.tif,C,5
2,2,010719SEL,010719VK3,vk3b002.tif,VK,5
3,3,010719SEL,010719VK3,vk3a000.tif,VK,1
4,4,010719SEL,010719VK7,vk7b000.tif,VK,1


In [98]:
control1 = tabulado[(tabulado['tratamiento']=='C') & (tabulado['frecuencia']== 1)].reset_index(drop=True)
control5 = tabulado[(tabulado['tratamiento']=='C') & (tabulado['frecuencia']== 5)].reset_index(drop=True)

In [110]:
tau_control1 = pd.DataFrame(columns = ['transient1','transient2','index'])
for j in range(0,len (control1)):
    celula = control1['célula'][j]
    foto = control1['foto'][j][:-4]
    for i in range(0,len (tau_file)):
        if (celula in tau_file['index'][i]) & (foto in tau_file['index'][i]):
            entry = tau_file.loc[i]
            tau_control1.loc[j] = entry

In [144]:
tau_control5 = pd.DataFrame(columns = ['transient1','transient2','index'])
for j in range(0,len (control5)):
    celula = control5['célula'][j]
    foto = control5['foto'][j][:-4]
    for i in range(0,len (tau_file)):
        if (celula in tau_file['index'][i]) & (foto in tau_file['index'][i]):
            entry = tau_file.loc[i]
            tau_control5.loc[j] = entry

In [113]:
def outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    idx = ~((column < (Q1 - 1.5 * IQR)) | (column > (Q3 + 1.5 * IQR)))
    return idx

In [114]:
tau_control1.describe()

,transient1,transient2
count,28.000000,28.000000
mean,113.399743,34.554246
std,422.374348,35.616544
min,1.170475,7.344018
25%,17.211528,17.039334
50%,19.355963,18.612546
75%,61.239167,45.642995
max,2263.965634,190.733388


In [118]:
tau_control1 = tau_control1[outliers(tau_control1) == True]
tau_control1.dropna()

,transient1,transient2
count,27.000000,27.000000
mean,33.749155,28.769834
std,28.130828,18.558848
min,1.170475,7.344018
25%,17.155162,17.036848
50%,18.928255,18.469751
75%,58.932724,43.045294
max,106.294554,77.159234


In [142]:
tau_control1 = tau_control1[tau_control1['transient1'] > 5]
tau_control1.describe()

,transient1,transient2
count,25.000000,25.000000
mean,36.223570,30.095987
std,27.766192,18.612969
min,13.452626,13.794829
25%,17.737692,17.103768
50%,19.783671,18.755342
75%,60.723462,45.418643
max,106.294554,77.159234


In [145]:
tau_control5 = tau_control5[(outliers(tau_control5) == True)]

In [146]:
tau_control5 = tau_control5.dropna()

In [147]:
tau_control5 = tau_control5[tau_control5['transient1'] > 5]

In [149]:
tau_control5.describe()

,transient1,transient2
count,27.000000,27.000000
mean,33.337443,28.092956
std,23.275818,16.494280
min,7.153172,5.723841
25%,16.979668,17.012539
50%,18.570233,18.896675
75%,45.945977,36.391429
max,82.656421,69.505792
